In [ ]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

In [ ]:
# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

In [ ]:
# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]

In [ ]:
# Given order
order = [
    "Актуализация", "Внебаланс (new)", "ДИЗ", "ДИЗ (возражения)", "ДИЗ (чередование)",
    "Коллекшн ранний сбор", "Матрица мотиваторов", "Матрица мотиваторов (распознавание эмоций)",
    "Матрица мотиваторов Хард. Ранний сбор", "Матрица мотиваторов Хард. Ранний сбор (кредитные карты)",
    "Матрица мотивации. Выезд", "Мультидоговоры. Универсальный", "Неконтактные", "Неконтактные (внебаланс)",
    "Неконтактные (поздний сбор)", "Неконтактные Суд", "Ожидание выезда", "Ожидание суда",
    "Подготовка к передаче в КА", "Преколлекшн (rename)", "Суд", "Усиление ПС Предцессия", 
    "Усиление позднего сбора", "ФЗ-230"
]
# Given column order
column_order = [
    "АО: Абонент не отвечает", "АО: Абонент недоступен", "АО: Номер не существует", 
    "АО: Нужен внутренний номер", "АО: Соединение установлено", "АО: Умные голосовые помощники", 
    "АО: Факс", "Должник молчит", "Должник не уверен", "Должник неизвестен", "Должник умер", 
    "Запрос реструктуризации", "Заявление о факте платежа", "Клиент болен", "Не передадут информацию", 
    "Не пройдена верификация", "Неконструктивный диалог", "Нужна помощь оператора", "Нужна помощь оператора *", 
    "Обещание оплатить", "Обещание частичной оплаты", "Оплата по испол.листу", "Отказ от верификации", 
    "Отказ от оплаты", "Отрицает долг", "Перезвонить", "Просьба передать информацию", "Сброс звонка роботом", 
    "Связь прервалась"
]

In [ ]:

# 1. Filter the dataframe
filtered_df = df[df['Результат автооценки'] != 100] # !!!
#filtered_df = df # !!!
# 2. Create the pivot table
pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='size', 
                                      fill_value=0)

'''pivot_table = filtered_df.pivot_table(index='Имя колл-листа', 
                                      columns='Результат робота', 
                                      values='Результат автооценки', 
                                      aggfunc='count', 
                                      fill_value=0)'''

pivot_table_mean = df.pivot_table(index='Имя колл-листа', columns='Результат робота', values='Результат автооценки', aggfunc='mean')


'''# 3. Ensure all desired columns are present
for col in column_order:
    if col not in pivot_table.columns:
        pivot_table[col] = 0

# Ensure all desired columns are present in the mean pivot table
for col in column_order:
    if col not in pivot_table_mean.columns:
        pivot_table_mean[col] = 0'''


'''# Reorder columns based on the given order
pivot_table = pivot_table[column_order]
pivot_table_mean = pivot_table_mean[column_order]
# Assuming pivot_table is the pivot table you previously created
pivot_table = pivot_table[column_order]
pivot_table = pivot_table.reindex(order)
pivot_table_mean = pivot_table_mean.reindex(order)'''

In [ ]:
pivot_table

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill
import xlsxwriter


def format_xlsx(pivot_table:pd.core.frame.DataFrame, sheet:str = 'Отчет 1', name:str = "pivot_table_gradient_colorscale.xlsx"):
    # Create an Excel writer and export the pivot table to an Excel file
    excel_file_path = name
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        pivot_table.to_excel(writer, sheet_name=sheet, index=True)

        # Get the xlsxwriter workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets[sheet]

        # Get the dimensions of the pivot table
        max_row = len(pivot_table)
        max_col = len(pivot_table.columns)

        # Add a format for the header cells
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'border': 1, 'bg_color': '#EFEFEF', 'align': 'center'})

        # Set the column width and format for the header
        for col_num, value in enumerate(pivot_table.columns.values):
            worksheet.write(0, col_num + 1, value, header_format)
            column_len = max(pivot_table[value].astype(str).str.len().max(), len(value)) + 2
            worksheet.set_column(col_num + 1, col_num + 1, column_len)
        
        # Add a format for the value cells
        value_format = workbook.add_format({'border': None, 'valign': 'vcenter', 'align': 'center',})

        # Get Max and Min of the value FFS
        max_value = pivot_table.max().max()
        min_value = pivot_table.min().min()


        # Apply gradient color scale to value cells
        worksheet.conditional_format(1, 1, max_row, max_col, {
            'type': '3_color_scale',
            'min_color': '#A6D86E',  # Green
            'mid_color': '#FFFFFE',  # White (for NaN)
            'max_color': '#e85f5f',  # Red
            'min_type': 'num',
            'mid_type': 'num',
            'max_type': 'num'
        })
        print(f'file: {name} -- Transformed 0')

In [ ]:
format_xlsx(pivot_table)

# Динамика по кол-листам + статусам + 3 в 1

In [ ]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv','./Reports/04-08.csv','./Reports/05-08.csv','./Reports/06-08.csv','./Reports/07-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
# Step 2: Filter rows where the duration is more than 0:00:30
df = df[df['Длительность звонка'] > pd.to_timedelta('0:00:10')]


In [ ]:
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)

In [ ]:
# Mask
df_mistakes = df[df['Результат автооценки'] != 100] # !!!

In [ ]:
# Add numbers to index
# Add numbers to index
def update_index(dataframe):
    new_index = [f'{i if len(str(i)) > 1 else f"0{i}"} {row}' for i, row in enumerate(dataframe.index)]
    dataframe.index = new_index
    return dataframe


# Create dynamic Mistakes count (1)
pivot_df_mistakes = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Ошибки', aggfunc='sum')
pivot_df_mistakes = pivot_df_mistakes.fillna(0)
pivot_df_mistakes = pivot_df_mistakes.replace(0.00, '')
pivot_df_mistakes.columns = pd.to_datetime(pivot_df_mistakes.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_mistakes = pivot_df_mistakes.sort_index(axis=1)  # Fix date Time
tmp_pivot_df_mistakes = pivot_df_mistakes.copy()  # Fix %%
pivot_df_mistakes.index = pivot_df_mistakes.index + ' (ошибки шт.)'
pivot_df_mistakes = update_index(pivot_df_mistakes)
# Create dynamic Calls count (2)
pivot_df_calls = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='count', fill_value=0)
pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
tmp_pivot_df_calls = pivot_df_calls.copy()  # Fix %%
pivot_df_calls.index = pivot_df_calls.index + ' (всего шт.)'
pivot_df_calls = update_index(pivot_df_calls)
# Create dynamic Mean Autoscore (3)
pivot_df_mean = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
pivot_df_mean.index = pivot_df_mean.index + ' (средняя АО)'
pivot_df_mean = update_index(pivot_df_mean)
# Create dynamic Error Percentage (4)
pivot_df_mistakes_filled = tmp_pivot_df_mistakes.replace('', 0)
pivot_df_error_rate = (pivot_df_mistakes_filled / tmp_pivot_df_calls).applymap(lambda x: x if not pd.isna(x) else '')
pivot_df_error_rate.index = pivot_df_error_rate.index + ' (доля ошибок %)'
pivot_df_error_rate = update_index(pivot_df_error_rate)
# Create Mega-Pivot
# Concatenate the pivot tables vertically along rows (axis=0)
mega_pivot = pd.concat([pivot_df_mistakes, pivot_df_calls, pivot_df_mean, pivot_df_error_rate], axis=0)
mega_pivot = mega_pivot.sort_index()


In [ ]:
error_rate_rows = mega_pivot[mega_pivot.index.str.contains("(доля ошибок %)")]
errors_rows = mega_pivot[mega_pivot.index.str.contains("(ошибки шт.)")]

In [ ]:
error_rate_rows.apply(pd.to_numeric, errors='coerce').max().max()
error_rate_rows.apply(pd.to_numeric, errors='coerce').min().min()

errors_rows.apply(pd.to_numeric, errors='coerce').max().max()
errors_rows.apply(pd.to_numeric, errors='coerce').min().min()

Excel Shit

In [ ]:
import pandas as pd

# Find min/max percent
max_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').max().max()
min_percent = error_rate_rows.apply(pd.to_numeric, errors='coerce').min().min()
# Find min/max mistakes
max_errors = errors_rows.apply(pd.to_numeric, errors='coerce').max().max()
min_errors = errors_rows.apply(pd.to_numeric, errors='coerce').min().min()
# Specify the Excel file path
excel_file_path = 'output.xlsx'

# Create a color scale conditional formatting rule
color_scale_rule_percent = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_percent,
                'mid_type': 'num',
                'mid_value': (max_percent-min_percent)/3,
                'max_type': 'num',
                'max_value': max_percent
                }

# Create a color scale conditional formatting rule
color_scale_rule_errors = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_errors,
                'mid_type': 'num',
                'mid_value': (max_errors-min_errors)/3,
                'max_type': 'num',
                'max_value': max_errors
                }

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

# Write the DataFrame to the Excel file
mega_pivot.to_excel(writer, sheet_name='Все звонки')

# Access the xlsxwriter workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Все звонки']

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_percent)
# Define a percentage format
percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        worksheet.set_row(row_num+1, None, percentage_format)


for row_num, row in enumerate(mega_pivot.index):
    if "(ошибки шт.)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_errors)

# Custom index format
index_format = workbook.add_format(
    {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
for row_num, row in enumerate(mega_pivot.index):
    worksheet.write(f'A{row_num+2}', row, index_format)

# Define a white fill format
white_fill_format = workbook.add_format({'bg_color': '#FFFFFF', 'border': 0})

# Apply the white background to the entire worksheet
worksheet.set_column(0, 0, 60, white_fill_format)
worksheet.set_column(1, 100, 18, white_fill_format)
'''# Custom header format
header_format = workbook.add_format(
                {'bold': True, 'border': 1, 'bg_color': '#AFEFEF','align': 'center'})
for col_num, value in enumerate(mega_pivot.columns.values):
                worksheet.write(0, col_num + 1, str(value)[:-19], header_format)'''
# Save the Excel file
writer.save()

# Отчет ошибки колл-лист + результат 

Pandas

In [ ]:
import pandas as pd

csv_list = ['./Reports/01-08.csv', './Reports/02-08.csv','./Reports/03-08.csv','./Reports/04-08.csv','./Reports/05-08.csv','./Reports/06-08.csv','./Reports/07-08.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])

In [ ]:
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)

In [ ]:
def create_multiindex(dataframe, sub_index:str):
    # Create MultiIndex
    multiindex = []
    for i, column in enumerate(dataframe):
        multiindex.append((column, sub_index))
    dataframe.columns = pd.MultiIndex.from_tuples(multiindex)
    return dataframe

# Calculate number of calls for each pair
pivot_df_calls = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Результат автооценки', aggfunc='count', fill_value='')
pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
# Calculate number of errors
pivot_df_errors = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Ошибки', aggfunc='sum', fill_value='')
pivot_df_errors.columns = pd.to_datetime(pivot_df_errors.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_errors = pivot_df_errors.sort_index(axis=1)  # Fix date Time
# Calculate mean autoscore
pivot_df_mean = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
# Calculate error rate
pivot_df_error_rate = (pivot_df_errors.replace("", pd.NA) / pivot_df_calls.replace("", pd.NA)).applymap(lambda x: x if not pd.isna(x) else '')
pivot_df_error_rate.columns = pd.to_datetime(pivot_df_error_rate.columns, format='%d.%m.%Y')  # Fix date Time
pivot_df_error_rate = pivot_df_error_rate.sort_index(axis=1)  # Fix date Time
max_error_rate = pivot_df_error_rate.apply(pd.to_numeric, errors='coerce').max().max()
min_error_rate = pivot_df_error_rate.apply(pd.to_numeric, errors='coerce').min().min()
# Create MultiIndex
pivot_df_calls = create_multiindex(pivot_df_calls, 'Зв.(шт.)')
pivot_df_errors = create_multiindex(pivot_df_errors, 'Ошб.(шт.)')
pivot_df_mean = create_multiindex(pivot_df_mean, 'Ср.АО')
pivot_df_error_rate = create_multiindex(pivot_df_error_rate, 'Ошб.%')

In [ ]:
# Create a list of the DataFrames you want to merge
dfs_to_merge = [pivot_df_calls, pivot_df_errors, pivot_df_mean, pivot_df_error_rate]

# Initialize an empty DataFrame with the same index as the original DataFrames
merged_df = pd.DataFrame(index=pivot_df_calls.index)

'''CREATE MULTIINDEX'''
multi_index = []
# Iterate through the DataFrames and concatenate their columns in the desired order
for num, column in enumerate(pivot_df_calls.columns):
        for dataframe in dfs_to_merge:
                #print(dataframe.iloc[:, num].name)
                #merged_df[dataframe.iloc[:, num].]
                col_name = (dataframe.iloc[:, num].name[0], dataframe.iloc[:, num].name[1])
                # Append the column name tuple to the list
                multi_index.append(col_name)
                merged_df[col_name] = dataframe.iloc[:, num]

merged_df.columns = pd.MultiIndex.from_tuples(multi_index)


In [ ]:
for i, j in enumerate(merged_df.head()):
    print(j[1], i+2)

Excel

In [ ]:
max_errors = max_error_rate  # kwarg
min_errors = min_error_rate  # kwarg
pivot_table = merged_df
name = 'Отчет 1'
sheet = 'Все звонки'

# Specify the Excel file path
excel_file_path = f'{name}.xlsx'

# Create a color scale conditional formatting rule
color_scale_rule_errors = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_errors,
                'mid_type': 'num',
                'mid_value': (max_errors-min_errors)/4,
                'max_type': 'num',
                'max_value': max_errors
                }

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

# Write the DataFrame to the Excel file
pivot_table.to_excel(writer, sheet_name=sheet)

# Access the xlsxwriter workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Все звонки']

'''for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_percent)
# Define a percentage format
percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        worksheet.set_row(row_num+1, None, percentage_format)


for row_num, row in enumerate(mega_pivot.index):
    if "(ошибки шт.)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_errors)

# Custom index format
index_format = workbook.add_format(
    {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
for row_num, row in enumerate(mega_pivot.index):
    worksheet.write(f'A{row_num+2}', row, index_format)
'''
# Define a white fill format
white_fill_format = workbook.add_format({'text_wrap': True, 'bg_color': '#FFFFFF', 'border': 0})
# Apply the white background to the entire worksheet
worksheet.set_column(0, 0, 25, white_fill_format)
worksheet.set_column(1, 1, 30, white_fill_format)
worksheet.set_column(2, 100, 9, white_fill_format)

percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF', 'border': 0})

for i, j in enumerate(pivot_table.head()):
    if j[1] == 'Ошб.%':
        worksheet.set_column(i+2, i+2, None, percentage_format)
        worksheet.conditional_format(0, i+2, 999, i+2, color_scale_rule_errors)

# Save the Excel file
writer.save()
    

# Страшный отчет: по блокам

Pandas

In [9]:
# 10.8 GB --> 20 GB --> 15 GB
import numpy as np
import pandas as pd

csv_list = ['./Reports/block_test.csv']
df = pd.DataFrame()

for i in csv_list:
    df_add = pd.read_csv(i, sep=';', header=0)
    df = pd.concat([df, df_add], ignore_index=True)

'''
№ п/п;ID звонка;Имя колл-листа;Результат робота;Дата звонка;Результат автооценки;Поисковый запрос: Ошибка идентификации: не уточняет дату рождения
'''

# Fix мультидоговоры
mask = df['№ п/п'].isna()
df = df[~mask]

# Step 1: Convert the 'Длительность звонка' column to Timedelta
#df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
df['Ошибки'] = df['Результат автооценки'] != 100
df['Дата'] = df['Дата звонка'].str.split(' ').str[0]
df = df.reset_index(drop=True)

# GET HEADERS AND REMOVE IRRELEVANT ONES
queries_list = list(df.columns)
columns_to_remove = [
    '№ п/п',
    'ID звонка',
    'Имя колл-листа',
    'Результат робота',
    'Дата звонка',
    'Результат автооценки',
    'Ошибки',
    'Дата',
    'Поисковый запрос: Все звонки, балл'
]
# Remove the specified columns
queries_list = [col for col in queries_list if col not in columns_to_remove]

# Все звонки fix
df['Всего звонков по листу'] = df['Поисковый запрос: Все звонки, балл']
queries_list.append('Всего звонков по листу')
#df['Всего звонков по листу'] = df['Всего звонков по листу'] * -1

for col in queries_list:
    df[col] = df[col] / df[col]

queries_list.append('Ошибки')

In [12]:
# First, melt the DataFrame to convert 'Запрос_1', 'Запрос_2', 'Запрос_3' into rows
melted_df = pd.melt(df, id_vars=['Имя колл-листа', 'Дата'], value_vars = queries_list, var_name='Запрос', value_name='Ошибки шт.')
melted_df.reset_index(drop=True)
del df

In [14]:

# Replace -1 with 1
#melted_df['Ошибки шт.'] = melted_df['Ошибки шт.'] * -1

# Now, create a pivot table to calculate sums
pivot_table = melted_df.pivot_table(
    values='Ошибки шт.',
    index=['Имя колл-листа', 'Запрос'],
    columns='Дата',
    aggfunc='sum'
)
del melted_df
# If you want to reset the index and have a cleaner view
pivot_table.reset_index()



In [ ]:
pivot_table

NameError: name 'pivot_table' is not defined

Excel

In [13]:
pivot_table = pivot_table
name = 'Отчет 4'
sheet = 'Все звонки'

# Specify the Excel file path
excel_file_path = f'{name}.xlsx'

# Create a color scale conditional formatting rule
'''color_scale_rule_errors = {
                'type': '3_color_scale',
                'min_color': '#A6D86E',  # Green
                'mid_color': '#FCFAA0',  # White (for NaN)
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': min_errors,
                'mid_type': 'num',
                'mid_value': (max_errors-min_errors)/4,
                'max_type': 'num',
                'max_value': max_errors
                }'''

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

# Write the DataFrame to the Excel file
pivot_table.to_excel(writer, sheet_name=sheet)

# Access the xlsxwriter workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Все звонки']

'''for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_percent)
# Define a percentage format
percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF'})

for row_num, row in enumerate(mega_pivot.index):
    if "(доля ошибок %)" in row:
        worksheet.set_row(row_num+1, None, percentage_format)


for row_num, row in enumerate(mega_pivot.index):
    if "(ошибки шт.)" in row:
        format_range = 'B' + str(row_num + 2) + ':AZ' + str(row_num + 2)  # Adjust column range as needed
        worksheet.conditional_format(format_range, color_scale_rule_errors)

# Custom index format
index_format = workbook.add_format(
    {'bold': True, 'border': 1, 'bg_color': '#FFFFFF', 'align': 'left'})
for row_num, row in enumerate(mega_pivot.index):
    worksheet.write(f'A{row_num+2}', row, index_format)
'''
# Define a white fill format
white_fill_format = workbook.add_format({'text_wrap': True, 'bg_color': '#FFFFFF', 'border': 0})
# Apply the white background to the entire worksheet
'''worksheet.set_column(0, 0, 25, white_fill_format)
worksheet.set_column(1, 1, 30, white_fill_format)'''
worksheet.set_column(0, 100, None, white_fill_format)

'''percentage_format = workbook.add_format({'num_format': '0.00%', 'bg_color': '#FFFFFF', 'border': 0})

for i, j in enumerate(pivot_table.head()):
    if j[1] == 'Ошб.%':
        worksheet.set_column(i+2, i+2, None, percentage_format)
        worksheet.conditional_format(0, i+2, 999, i+2, color_scale_rule_errors)'''

# Save the Excel file
writer.save()

In [ ]:
worksheet.col

# Отчет Блок

Pandas